In [19]:
import pandas as pd
import numpy as np
import Levenshtein

# Charger les données
df = pd.read_csv("random_1000_clusters.csv")

# Nettoyage
df = df[df['full_text'].notna()].copy()
df

,source,title,url,published_at,tags,content,full_text,timestamp,cluster,pca_1,pca_2
0,Sky_News_Arabic,دراسة تحذر: إنفلونزا الطيور قد تصيب عمال المزا...,https://www.skynewsarabia.com/technology/1753538,2024-11-07,صحة,دعا مسؤولو قطاع الصحة الفيدرالية يوم الخميس إل...,دراسة تحذر: إنفلونزا الطيور قد تصيب عمال المزا...,1730937600,6,-12.439412,-3.969779
1,Sky_News_Arabic,بينيت: لهذا السبب علينا الاستيلاء على رفح,https://www.skynewsarabia.com/middle-east/1699710,2024-03-14,شرق أوسط,قال رئيس الوزراء الإسرائيلي السابق نفتالي بيني...,بينيت: لهذا السبب علينا الاستيلاء على رفح قال ...,1710374400,10,-17.525290,-9.870685
2,Al Hurra,أمازيغية ومساجد وأخلاق.. قضايا أشعلت منصات الت...,https://www.alhurra.com/maghreb-voices/2024/11...,2024-11-01,no_tags,أثارت زيارة وزيرة الثقافة الفرنسية، رشيدة داتي...,أمازيغية ومساجد وأخلاق.. قضايا أشعلت منصات الت...,1730419200,8,20.630802,-0.571323
3,France_24,غزة: تحركات الجامعات الأميركية تحدٍ انتخابي لب...,https://www.france24.com/ar/%D9%81%D9%8A%D8%AF...,2024-04-26,الأخبار المغاربية,ما الذي تقوله التحركات الطلابية التي تتسع في ا...,غزة: تحركات الجامعات الأميركية تحدٍ انتخابي لب...,1714089600,10,-17.191772,-10.327633
4,France_24,فرحة سعودية باستضافة كأس العالم 2034 يقابلها ق...,https://www.france24.com/ar/%D8%B1%D9%8A%D8%A7...,2024-12-13,رياضة,شهدت الرياض أجواء احتفالية مبهجة بعد فوز السعو...,فرحة سعودية باستضافة كأس العالم 2034 يقابلها ق...,1734048000,6,-9.878988,-2.507187
...,...,...,...,...,...,...,...,...,...,...,...
995,Sky_News_Arabic,"صفقة ""هيس"" تشعل صراعا بين عمالقة النفط في الول...",https://www.snabusiness.com/article/1717188,2024-05-27,أخبار الشركات,"بعد أيام من إتمام استحواذ شركة ""شيفرون"" الأمير...","صفقة ""هيس"" تشعل صراعا بين عمالقة النفط في الول...",1716768000,12,-9.285624,10.812364
996,Al Hurra,"بعد ""معلومات خاطئة"".. ماكرون يتحدث عن توقيف مؤ...",https://www.alhurra.com/arabic-and-internation...,2024-08-26,no_tags,قال الرئيس الفرنسي إيمانويل ماكرون، الاثنين، إ...,"بعد ""معلومات خاطئة"".. ماكرون يتحدث عن توقيف مؤ...",1724630400,8,15.850406,0.332709
997,France_24,المخرج الأمريكي كوبولا يطمح إلى الظفر بسعفة ذه...,https://www.france24.com/ar/%D8%AB%D9%82%D8%A7...,2024-05-16,ثقافة,يسعى المخرج الأمريكي فرانسيس فورد كوبولا إلى ا...,المخرج الأمريكي كوبولا يطمح إلى الظفر بسعفة ذه...,1715817600,6,-8.203756,-4.088220
998,Al Hurra,الاقتصاد الإسرائيلي في الحرب,https://www.alhurra.com/israel/2024/10/06/%D8%...,2024-10-06,إسرائيل الإسرائيلي,تمكن الاقتصاد الإسرائيلي على مدى عام تقريبا من...,الاقتصاد الإسرائيلي في الحرب تمكن الاقتصاد الإ...,1728172800,9,17.559583,0.507874


In [ ]:
# Choisir un cluster (par exemple 0)
for k in range(18) :
    cluster_id = k
    text = df[df['cluster'] == cluster_id]['full_text'].tolist()
    
    # Calcul des similarités Levenshtein normalisées
    similarities = []
    n = len(text)
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = Levenshtein.distance(text[i], text[j])
            max_len = max(len(text[i]), len(text[j]))
            sim = 1 - dist / max_len if max_len > 0 else 1
            similarities.append(sim)
    
    # Résultat
    mean_similarity = np.mean(similarities) if similarities else 1.0
    print(f"Nombre de titres dans le cluster {cluster_id} : {len(text)}")
    print(f"Similarité moyenne (Levenshtein) : {mean_similarity:.4f}")

Nombre de titres dans le cluster 0 : 28
Similarité moyenne (Levenshtein) : 0.2501
Nombre de titres dans le cluster 1 : 96
Similarité moyenne (Levenshtein) : 0.2578
Nombre de titres dans le cluster 2 : 36
Similarité moyenne (Levenshtein) : 0.2718
Nombre de titres dans le cluster 3 : 63
Similarité moyenne (Levenshtein) : 0.2494
Nombre de titres dans le cluster 4 : 23
Similarité moyenne (Levenshtein) : 0.2445
Nombre de titres dans le cluster 5 : 17
Similarité moyenne (Levenshtein) : 0.2548
Nombre de titres dans le cluster 6 : 84
Similarité moyenne (Levenshtein) : 0.2528
Nombre de titres dans le cluster 7 : 52
Similarité moyenne (Levenshtein) : 0.2686
Nombre de titres dans le cluster 8 : 213
Similarité moyenne (Levenshtein) : 0.2499


In [25]:
df['cluster'].value_counts()

cluster
8     213
9     110
1      96
6      84
10     70
3      63
7      52
11     41
15     39
2      36
13     32
16     29
0      28
14     27
4      23
12     21
17     19
5      17
Name: count, dtype: int64